In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import eqmarl

2025-01-26 12:01:14.087239: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/current/lib64/slurm:/cm/shared/apps/slurm/current/lib64
2025-01-26 12:01:14.087277: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-01-26 12:01:18.528576: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/current/lib64/slurm:/cm/shared/apps/slurm/current/lib64
2025-01-26 12:01:18.528604: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2025-01-26 12:01:18.528623: I tensorflow/stream_executor/cuda/cuda_diagnost

In [3]:
import os
import gymnasium as gym
import minigrid # Must import minigrid after gymnasium to register the environments properly.
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras

In [4]:
gym.vector.make

<function gymnasium.vector.make(id: str, num_envs: int = 1, asynchronous: bool = True, wrappers: Union[Callable[[gymnasium.core.Env], gymnasium.core.Env], List[Callable[[gymnasium.core.Env], gymnasium.core.Env]], NoneType] = None, disable_env_checker: Optional[bool] = None, **kwargs) -> gymnasium.vector.vector_env.VectorEnv>

In [5]:
env_id = "MiniGrid-LavaCrossingS9N1-v0"
env = gym.make(env_id, render_mode="rgb_array")

trigger = lambda t: t % 10 == 0
env = gym.wrappers.RecordVideo(env, video_folder="./env_render_video", episode_trigger=trigger, disable_logger=False)

for i in range(50):
    termination, truncation = False, False
    _ = env.reset(seed=123)
    while not (termination or truncation):
        obs, reward, termination, truncation, info = env.step(env.action_space.sample())
env.close()

MoviePy - Building video /home/acd1797/testingtobedeleted/eqmarl/env_render_video/rl-video-episode-0.mp4.
MoviePy - Writing video /home/acd1797/testingtobedeleted/eqmarl/env_render_video/rl-video-episode-0.mp4



MoviePy - Done !
MoviePy - video ready /home/acd1797/testingtobedeleted/eqmarl/env_render_video/rl-video-episode-0.mp4
MoviePy - Building video /home/acd1797/testingtobedeleted/eqmarl/env_render_video/rl-video-episode-10.mp4.
MoviePy - Writing video /home/acd1797/testingtobedeleted/eqmarl/env_render_video/rl-video-episode-10.mp4



MoviePy - Done !
MoviePy - video ready /home/acd1797/testingtobedeleted/eqmarl/env_render_video/rl-video-episode-10.mp4


MoviePy - Building video /home/acd1797/testingtobedeleted/eqmarl/env_render_video/rl-video-episode-20.mp4.
MoviePy - Writing video /home/acd1797/testingtobedeleted/eqmarl/env_render_video/rl-video-episode-20.mp4



MoviePy - Done !
MoviePy - video ready /home/acd1797/testingtobedeleted/eqmarl/env_render_video/rl-video-episode-20.mp4


...

MoviePy - Done !
MoviePy - video ready /home/acd1797/testingtobedeleted/eqmarl/env_render_video/rl-video-episode-30.mp4


MoviePy - Building video /home/acd1797/testingtobedeleted/eqmarl/env_render_video/rl-video-episode-40.mp4.
MoviePy - Writing video /home/acd1797/testingtobedeleted/eqmarl/env_render_video/rl-video-episode-40.mp4



MoviePy - Done !
MoviePy - video ready /home/acd1797/testingtobedeleted/eqmarl/env_render_video/rl-video-episode-40.mp4


MoviePy - Building video /home/acd1797/testingtobedeleted/eqmarl/env_render_video/rl-video-episode-50.mp4.
MoviePy - Writing video /home/acd1797/testingtobedeleted/eqmarl/env_render_video/rl-video-episode-50.mp4



MoviePy - Done !
MoviePy - video ready /home/acd1797/testingtobedeleted/eqmarl/env_render_video/rl-video-episode-50.mp4


In [11]:
import minigrid.wrappers

env_id = "MiniGrid-LavaCrossingS9N1-v0"
env = gym.vector.make(env_id, num_envs=2, wrappers=[
    minigrid.wrappers.ImgObsWrapper,
    eqmarl.wrappers.StepRewardWrapper,
    minigrid.wrappers.ActionBonus,
    ], render_mode="rgb_array",
    )
env.render_mode = "rgb_array"

env.envs

# trigger = lambda t: t % 10 == 0
# env = gym.wrappers.RecordVideo(env, video_folder="./env_render_video_vec", episode_trigger=trigger, disable_logger=False)
# print(env.render_mode)
# print(env.is_vector_env)

# for i in range(50):
#     termination, truncation = False, False
#     _ = env.reset(seed=123)
#     while not (termination or truncation):
#         obs, reward, termination, truncation, info = env.step(env.action_space.sample())
# env.close()

/home/acd1797/.conda/envs/eqmarltesting/lib/python3.9/site-packages/gymnasium/vector/__init__.py:53: UserWarning: WARN: `gymnasium.vector.make(...)` is deprecated and will be replaced by `gymnasium.make_vec(...)` in v1.0
  gym.logger.warn(


AttributeError: 'AsyncVectorEnv' object has no attribute 'envs'

In [31]:
class CustomRecordVideo(gym.wrappers.RecordVideo):
    
    def reset(self, **kwargs):
        """Reset the environment using kwargs and then starts recording if video enabled."""
        observations = super().reset(**kwargs)
        self.terminated = False
        self.truncated = False
        
        # If already recording, then save existing video and start recording again.
        if self.recording:
            assert self.video_recorder is not None
            self.close_video_recorder()

        if self._video_enabled():
            self.start_video_recorder()
        return observations

In [32]:
env_id = "MiniGrid-LavaCrossingS9N1-v0"
num_envs = 2

def create_env(env_id: str, env_idx: int, **kwargs):

    def _make_env():
        env = gym.make(env_id, **kwargs)
        
        print(f"{env_idx=}")
        
        # Wrap.
        env = minigrid.wrappers.ImgObsWrapper(env)
        env = eqmarl.wrappers.StepRewardWrapper(env)
        env = minigrid.wrappers.ActionBonus(env)

        # Video wrapper.
        trigger = lambda t: t % 10 == 0
        # env = gym.wrappers.RecordVideo(
        env = CustomRecordVideo(
            env=env,
            video_folder=f"./env_render_video_vec/env-{env_idx}",
            name_prefix=f"env-{env_idx}",
            episode_trigger=trigger,
            disable_logger=False,
            )
        return env

    return _make_env
    
env = gym.vector.AsyncVectorEnv([
    create_env(env_id, env_idx=i, render_mode="rgb_array")
    for i in range(num_envs)
])


for i in range(3):
    termination, truncation = [False]*num_envs, [False]*num_envs
    _ = env.reset(seed=123)
    for t in range(500):
        obs, reward, termination, truncation, info = env.step(env.action_space.sample())
        print(f"{termination=}")
        print(f"{truncation=}")
        if any(termination) or any(truncation):
            break
_ = env.reset(seed=123)
env.close()

env_idx=0
env_idx=0


/home/acd1797/.conda/envs/eqmarltesting/lib/python3.9/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /home/acd1797/testingtobedeleted/eqmarl/env_render_video_vec/env-0 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


env_idx=1
MoviePy - Building video /home/acd1797/testingtobedeleted/eqmarl/env_render_video_vec/env-0/env-0-episode-0.mp4.
MoviePy - Building video /home/acd1797/testingtobedeleted/eqmarl/env_render_video_vec/env-1/env-1-episode-0.mp4.
MoviePy - Writing video /home/acd1797/testingtobedeleted/eqmarl/env_render_video_vec/env-0/env-0-episode-0.mp4

MoviePy - Writing video /home/acd1797/testingtobedeleted/eqmarl/env_render_video_vec/env-1/env-1-episode-0.mp4



MoviePy - Done !
MoviePy - video ready /home/acd1797/testingtobedeleted/eqmarl/env_render_video_vec/env-1/env-1-episode-0.mp4
MoviePy - Done !
MoviePy - video ready /home/acd1797/testingtobedeleted/eqmarl/env_render_video_vec/env-0/env-0-episode-0.mp4
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination

MoviePy - Done !
MoviePy - video ready /home/acd1797/testingtobedeleted/eqmarl/env_render_video_vec/env-0/env-0-episode-0.mp4
MoviePy - Building video /home/acd1797/testingtobedeleted/eqmarl/env_render_video_vec/env-1/env-1-episode-0.mp4.
MoviePy - Writing video /home/acd1797/testingtobedeleted/eqmarl/env_render_video_vec/env-1/env-1-episode-0.mp4

termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=a

MoviePy - Done !
MoviePy - video ready /home/acd1797/testingtobedeleted/eqmarl/env_render_video_vec/env-1/env-1-episode-0.mp4
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
ter

MoviePy - Done !
MoviePy - video ready /home/acd1797/testingtobedeleted/eqmarl/env_render_video_vec/env-1/env-1-episode-0.mp4
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
termination=array([False, False])
truncation=array([False, False])
ter

Examples - Run the environment for 50 episodes, and save the video every 10 episodes starting from the 0th:
        >>> import os
        >>> import gymnasium as gym
        >>> env = gym.make("LunarLander-v3", render_mode="rgb_array")
        >>> trigger = lambda t: t % 10 == 0
        >>> env = RecordVideo(env, video_folder="./save_videos1", episode_trigger=trigger, disable_logger=True)
        >>> for i in range(50):
        ...     termination, truncation = False, False
        ...     _ = env.reset(seed=123)
        ...     while not (termination or truncation):
        ...         obs, rew, termination, truncation, info = env.step(env.action_space.sample())
        ...
        >>> env.close()
        >>> len(os.listdir("./save_videos1"))
        5

In [100]:
from typing import Callable, Optional
from gymnasium import logger
from gymnasium.vector import VectorEnv, VectorEnvWrapper
from gymnasium.utils import RecordConstructorArgs
from gymnasium.wrappers.record_video import capped_cubic_video_schedule
from gymnasium.wrappers.monitoring import video_recorder
from moviepy.video.io.ImageSequenceClip import ImageSequenceClip

class VectorRecordVideo(VectorEnvWrapper, RecordConstructorArgs):
    """Records videos for all sub-environments of a parent vector environment.
    
    Equivalent of class ``gymnasium.wrappers.RecordVideo`` for vectorized environments.
    
    ``video_folder``
    |- env-0-``name_prefix``-epispde-0.mp4
    |- env-0-``name_prefix``-epispde-1.mp4
    |- env-1-``name_prefix``-epispde-0.mp4
    |- env-1-``name_prefix``-epispde-1.mp4
    |- ...
    """
    def __init__(
        self,
        env: VectorEnv,
        video_folder: str,
        episode_trigger: Callable[[int], bool] = None,
        step_trigger: Callable[[int], bool] = None,
        video_length: int = 0,
        name_prefix: str = "rl-video",
        disable_logger: bool = False,
        render_fps: int = 30,
        video_ext: str = 'mp4',
    ):
        RecordConstructorArgs.__init__(
            self,
            video_folder=video_folder,
            episode_trigger=episode_trigger,
            step_trigger=step_trigger,
            video_length=video_length,
            name_prefix=name_prefix,
            disable_logger=disable_logger,
        )
        VectorEnvWrapper.__init__(self, env)
        
        if env.render_mode in {None, "human", "ansi", "ansi_list"}:
            raise ValueError(
                f"Render mode is {env.render_mode}, which is incompatible with"
                f" VectorRecordVideo. Initialize your environment with a render_mode"
                f" that returns an image, such as rgb_array."
            )
        
        if episode_trigger is None and step_trigger is None:
            episode_trigger = capped_cubic_video_schedule

        trigger_count = sum(x is not None for x in [episode_trigger, step_trigger])
        assert trigger_count == 1, "Must specify exactly one trigger"

        self.name_prefix = name_prefix
        self.video_length = video_length
        self.episode_trigger = episode_trigger
        self.step_trigger = step_trigger
        self.disable_logger = disable_logger
        # self.video_recorders: list[Optional[video_recorder.VideoRecorder]] = [None] * self.num_envs
        self.captured_frames: list[list[np.ndarray]] = []
        self.recording: bool = False
        # self.recording: list[bool] = [False] * self.num_envs
        # self.recorded_frames: list[int] = [0] * self.num_envs
        self.recorded_frames: int = 0
        self.step_id = 0
        self.episode_id = -1
        
        self.frames_per_sec = self.env.metadata.get("render_fps", render_fps)
        assert video_ext in ('mp4', 'avi', 'ogv', 'webm'), "Unspported video extension, must be one of ('mp4', 'avi', 'ogv', 'webm')."
        self.video_ext = video_ext

        self.video_folder = os.path.abspath(video_folder)
        # Create output folder if needed
        if os.path.isdir(self.video_folder):
            logger.warn(
                f"Overwriting existing videos at {self.video_folder} folder "
                f"(try specifying a different `video_folder` for the `VectorRecordVideo` wrapper if this is not desired)"
            )
        os.makedirs(self.video_folder, exist_ok=True)
    
    def _video_enabled(self):
        if self.step_trigger:
            return self.step_trigger(self.step_id)
        else:
            return self.episode_trigger(self.episode_id)
    
    def reset(self, **kwargs):
        observations, infos = super().reset(**kwargs)
        
        self.episode_id += 1

        # If already recording, then save any frames to a new video.
        if self.recording:
            self.stop_video()
        # Start a new video if necessary.
        if self._video_enabled():
            self.start_video()
        return observations, infos
    
    def capture_frame(self):
        
        frames = self.env.call("render")
        self.captured_frames.append(frames)
        self.recorded_frames += 1
        # print(f"{self.recorded_frames=}, {frames=}")
    
    
    def step(self, actions):
        (
            observations,
            rewards,
            terminateds,
            truncateds,
            infos,
        ) = self.env.step(actions)
        
        self.step_id += 1
        # if any(terminateds) or any(truncateds):
        #     self.episode_id += 1
        
        if self.recording:
            self.capture_frame()
            
            # Stop video if it is too long.
            if self.video_length > 0:
                if self.recorded_frames > self.video_length:
                    self.stop_video()
            
            # Stop video if any sub-environment has either terminated or truncated.
            # This ensures that the videos for all sub-environments are synchronized.
            elif any(terminateds) or any(truncateds):
                self.stop_video()

        # Start a new recording session if necessary.
        elif self._video_enabled():
            self.start_video()

        return observations, rewards, terminateds, truncateds, infos
    
    def stop_video(self):
        print(f"STOP")

        # Do nothing if not recording.
        if not self.recording:
            return

        if self.recorded_frames > 0:
            # Convert frames to Numpy matrix.
            # Shape is (time, env, *img_shape).
            frames = np.array(self.captured_frames)
            
            # Generate video clip for all frames of each environment.
            for i in range(self.num_envs):

                video_name = f"env-{i}-{self.video_name_postfix}"
                video_file = f"{video_name}.{self.video_ext}"
                path = os.path.join(self.video_folder, video_file)
                clip = ImageSequenceClip(list(frames[:,i]), fps=self.frames_per_sec)
                moviepy_logger = None if self.disable_logger else "bar"
                clip.write_videofile(path, logger=moviepy_logger)

        self.recording = False
    
    def start_video(self):
        print(f"START")
        self.captured_frames = []
        self.recorded_frames = 0
        self.recording = True
        
        
        self.video_name_postfix = f"step-{self.step_id}"
        if self.episode_trigger:
            self.video_name_postfix = f"episode-{self.episode_id}"
    
    def close(self):
        super().close()
        self.stop_video()

In [102]:
import minigrid.wrappers

env_id = "MiniGrid-LavaCrossingS9N1-v0"
env = gym.vector.make(env_id, num_envs=2, wrappers=[
    minigrid.wrappers.ImgObsWrapper,
    eqmarl.wrappers.StepRewardWrapper,
    minigrid.wrappers.ActionBonus,
    ], render_mode="rgb_array",
    )
env.render_mode = "rgb_array"

trigger = lambda t: t % 10 == 0
env = VectorRecordVideo(
    env=env,
    video_folder="./env_render_video_vec",
    episode_trigger=trigger,
    # step_trigger=trigger,
    # video_length=5,
    disable_logger=True,
    )

for i in range(31):
    termination, truncation = False, False
    _ = env.reset(seed=123)
    for t in range(50):
        obs, reward, termination, truncation, info = env.step(env.action_space.sample())
        if any(termination) or any(truncation):
            break
    print(f"{t=}")
env.close()

/home/acd1797/.conda/envs/eqmarltesting/lib/python3.9/site-packages/gymnasium/vector/__init__.py:53: UserWarning: WARN: `gymnasium.vector.make(...)` is deprecated and will be replaced by `gymnasium.make_vec(...)` in v1.0
  gym.logger.warn(
/localscratch/2905540/ipykernel_2584321/2504750628.py:78: UserWarning: WARN: Overwriting existing videos at /home/acd1797/testingtobedeleted/eqmarl/env_render_video_vec folder (try specifying a different `video_folder` for the `VectorRecordVideo` wrapper if this is not desired)
  logger.warn(
/home/acd1797/.conda/envs/eqmarltesting/lib/python3.9/site-packages/gymnasium/vector/vector_env.py:389: UserWarning: WARN: env.action_space to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.action_space` for environment variables.
  logger.warn(
/home/acd1797/.conda/envs/eqmarltesting/lib/python3.9/site-packages/gymnasium/vector/vector_env.py:389: UserWarning: WARN: env.num_envs to get 

START
STOP
t=47
t=31
t=49
t=49
t=26
t=49
t=49
t=49
t=49
t=49
START
STOP
t=41
t=49
t=49
t=49
t=49
t=49
t=49
t=49
t=49
t=49
START
t=49
STOP
t=49
t=49
t=49
t=49
t=45
t=49
t=49
t=49
t=49
START
STOP
t=18
STOP
